# Author and Institution Networks 
This will be a data exploration of the networks which arrise when we explore the realtionships between and around authors and institiues in health infomatics

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pyvis.network import Network 
from IPython.display import display

hello
